# EDA Times series

Reading Times Series Forecasting in Python by @Marco Peixeiro

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from scipy import stats
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import seaborn as sns


import ipywidgets as widgets

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

from datetime import date
import holidays
import calendar
import dateutil.easter as easter

from collections import defaultdict
le = defaultdict(LabelEncoder)

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(12, 8))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import gc
import os
import math
import random

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Admin
ID = "row_id"            # Id id x X index
INPUT = "../input/tabular-playground-series-jan-2022"

# time series data common new feature
DATE = "date"
YEAR = "year"
QUARTER = "quarter"
MONTH = "month"
WEEK = "week"
DAY = "day"
DAYOFYEAR = "dayofyear"
WEEKOFYEAR = "weekofyear"
DAYOFMONTH = "dayofMonth"
DAYOFWEEK = "dayofweek"
WEEKDAY = "weekday"

In [ ]:
from pathlib import Path


def load_data():
    # Read data
    data_dir = Path(INPUT)
    df_train = pd.read_csv(data_dir / "train.csv", parse_dates=[DATE],
                    usecols=['date', 'country', 'store', 'product', 'num_sold'],
                    dtype={
                        'country': 'category',
                        'store': 'category',
                        'product': 'category',
                        'num_sold': 'float64',
                    },
                    infer_datetime_format=True,)
    df_test = pd.read_csv(data_dir / "test.csv", index_col=ID, parse_dates=[DATE])
    column_y = df_train.columns.difference(
        df_test.columns)[0]  # column_y target_col label_col
    df_train[DATE] = pd.to_datetime(df_train[DATE])
    df_test[DATE] = pd.to_datetime(df_test[DATE])
    return df_train, df_test, column_y


In [ ]:
def process_data(df_train, df_test):
    # Preprocessing
    if FEATURE_ENGINEERING:
        df_train = feature_engineer(df_train)
        df_test = feature_engineer(df_test)

    return df_train, df_test

# Load Data
The first step is to gather the data. Then, we test for stationarity. In the event
where our series is not stationary, we apply transformations, such as differencing, until the
series is stationary. Then, we plot the ACF and look for significant autocorrelation
coefficients. In the case of a random walk, we will not see significant coefficients after lag 0.
On the other hand, if we see significant coefficients, then we must check if they become
abruptly non-significant after some lag q . If that is the case, then we know that we have a
moving average process of order q . Otherwise, we must follow a different set of steps to
discover the underlying process of our time series.

In [ ]:
train_df, test_df, column_y = load_data()

In [ ]:
train_df

In [ ]:
df = train_df.groupby(['country', 'store', 'product', 'date']).mean().unstack(['country', 'store', 'product']) #.loc['2015']

In [ ]:
train_subset = train_df[(train_df.country == 'Norway') & (train_df.store == 'KaggleMart') & (train_df['product'] == 'Kaggle Hat')].copy()

In [ ]:
train_subset

## Plot of subset year 2015-2018 Norway KaggleMart Kaggle Hat

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(train_subset[column_y])
ax.set_xlabel('Time')
ax.set_ylabel('Kaggle Sales')
fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
train_subset.index = pd.DatetimeIndex(train_subset.date).to_period('D')

In [ ]:
from statsmodels.tsa.stattools import adfuller
ADF_result = adfuller(train_df[column_y])
print(f'ADF Statistic: {ADF_result[0]}')
print(f'p-value: {ADF_result[1]}')

The next step is to test for stationarity. We intuitively know that the series is not stationary
since there is an observable trend till 2018. Still, we will use the ADF test to make
sure. Again, we use the adfuller function from the statsmodels library and extract the ADF
statistic and p-value. If the ADF statistic is a large negative number and the p-value is
smaller than 0.05, then our series is stationary. Otherwise, we must apply transformations.

In [ ]:
ADF_result = adfuller(train_subset[column_y])
print(f'ADF Statistic: {ADF_result[0]}')
print(f'p-value: {ADF_result[1]}')

This results in an ADF statistic of -2.5 and a p-value of 0.11. Here, the ADF statistic is not a
large negative number and the p-value is greater than 0.05. Therefore, our time series is not
stationary and we must apply transformations to make it stationary.
In order to make our series stationary, we will try to stabilize the trend by applying a
first-order differencing. We can do so by using the diff method from the numpy library.
Remember that this method takes in a parameter n that specifies the order of differencing.
In this case, because it is a first-order differencing, n will be equal to 1.

In [ ]:
kaggle_2015_sales_diff = np.diff(train_subset[column_y], n=1)

In [ ]:
fig, ax = plt.subplots()
ax.plot(kaggle_2015_sales_diff)
ax.set_xlabel('Differenced volume of Kaggle sales. Notice how the trend component was stabilized since values')
ax.set_ylabel('Kaggle Sales')
fig.autofmt_xdate()
plt.tight_layout()

With a transformation applied to our series, we can test for stationarity again using the ADF
test. This time, make sure to run the test on the differenced data stored in the
*kaggle_2015_sales_diff* variable.

In [ ]:
ADF_result = adfuller(kaggle_2015_sales_diff)
print(f'ADF Statistic: {ADF_result[0]}')
print(f'p-value: {ADF_result[1]}')

In [ ]:
kaggle_2015_sales_diff2 = np.diff(kaggle_2015_sales_diff, n=1)
ADF_result = adfuller(kaggle_2015_sales_diff2)
print(f'ADF Statistic: {ADF_result[0]}')
print(f'p-value: {ADF_result[1]}')

This gives an ADF statistic of -10.5 and a p-value of 9.7 × 10 −19 . Therefore, with a large
negative ADF statistic and a p-value much smaller than 0.05, we can say that our series is
stationary.
Our next step is to plot the autocorrelation function. The statsmodels library
conveniently includes the plot_acf function for us. We simply pass in our differenced series
and specify the number of lags in the lags parameter. Remember that the number of lags
determines the range of values on the x-axis.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(kaggle_2015_sales_diff, lags=31);
plt.tight_layout()

- There are significant autocorrelation coefficients after lag 0. Therefore, it is not a random walk.
- The coefficients do not become abruptly non-significant after a certain lag, which means that it is not a purely moving average process either.
- We notice that there are significant coefficients after lag 0. In fact, they are significant up until lag 30.
- Notice the coefficients 7, 14, 21, 28(weekly, weekend). 
- Negative coefficients on weekdays Monday-Thursday.
- Non-significant coefficients remain in the shaded area of the plot.

Since we have significant autocorrelation coefficients inteval of 7, assumming that we have
a stationary moving average process of order 7. Therefore, we can use a 7-order
moving average model, or MA(7) model, to forecast our stationary time series.

In [ ]:
df_diff = pd.DataFrame({'kaggle_2015_sales_diff': kaggle_2015_sales_diff})
train = df_diff[:int(0.9*len(df_diff))]
test = df_diff[int(0.9*len(df_diff)):]
print(len(train))
print(len(test))

For the forecast horizon the moving average model bring in a particularity. The MA(q) model
does not allow us to forecast 50 steps into future in one shot. Remember that the moving
average model is linearly dependent on past error terms. Those terms are not observed in
the dataset and must therefore be recursively estimated. This means that for an MA(q)
model, we can only forecast q steps into the future. Any prediction made beyond that point
will not have past error terms and the model will only predict the mean. Therefore, there is
no added value in forecasting beyond q steps into the future because the predictions will fall
flat, as only the mean is returned, which is equivalent to a baseline model.

# Opps, this is for year 2019 365 days one-shot.
I stop here for now. :S

## Partial autocorrelation
Partial autocorrelation measures the correlation between lagged values in a time series when we remove the influence of other correlated lagged values. We can plot the partial autocorrelation function to determine the order of a stationary AR(p) process. The coefficients will be non-significant after lag *p*.

Let’s plot the PACF and see if the coefficients become abruptly non-significant after lag 2. If that is the case, then we know that we can use the PACF plot to determine the order of a stationary autoregressive process, just like we can use the ACF plot to determine the order of a stationary moving average process.

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(train_subset[column_y], lags=20) #train_subset[column_y] kaggle_2015_sales_diff
plt.tight_layout()

# Chapter 2

Previously, we covered the moving average process, also denoted as MA(q), where q is the order. We learned that in a moving average process, the present value is linearly dependent on current and past error terms. Therefore, if we predict more than q steps ahead, the prediction will fall flat and will return only the mean of the series because the error terms are not observed in the data and must be recursively estimated. Finally, we saw that we can determine the order of a stationary MA(q) process by studying the ACF plot;
the autocorrelation coefficients will be significant up until lag q. In the case where the autocorrelation coefficients slowly decay or exhibit a sinusoidal pattern, then we are possibly in the presence of an autoregressive process.

Here, we will first define the autoregressive process. Then, we will define the partial autocorrelation function and use it to find the order of the underlying autoregressive process of our dataset. Finally, we will use the AR(p) model to produce forecasts.

# Autoregressive process
An autoregressive process is a regression of a variable against itself. In time series, this means that the present value is linearly dependent on its past values.

The autoregressive process is denoted as AR(p) where p is the order. The general expression of an AR(p) model is:
y t = C + ϕ 1 y t −1 + ϵ t + ϕ 2 y t −2 + ϵ t +⋅⋅⋅ + ϕ p y t −p + ϵ

The next step is learning how to treat time series where you cannot infer an order from the ACF plot nor from the PACF plot. This means that both figures exhibit a slowly decaying pattern or a sinusoidal pattern. In such case, we are in the presence of an autoregressive moving average process or ARMA. This denotes the combination of both the autoregressive and moving average processes.

We will examine the autoregressive moving average process or ARMA(p,q), where p denotes the order of the autoregressive portion, and q denotes the order of the moving average portion. Furthermore, we will not be able to use the ACF and PACF plots to determine the orders q and p respectively, as both plots will show either a slowly decaying or sinusoidal pattern. Thus, we will define a general modeling procedure that will
allow us to model such complex time series. This procedure involves model selection using the Akaike’s Information Criterion or AIC, which will determine the optimal combination of p and q for our series. Then, we must evaluate the models’ validity using residual analysis, by studying the correlogram, Q-Q plot and density plot of the model’s residuals in order to assess if they closely resemble white noise. If that is the case, we can move on to forecasting our time series using the ARMA(p,q) model.

In [ ]:
from statsmodels.tsa.arima_process import ArmaProcess
import numpy as np
np.random.seed(42)
ar1 = np.array([1, -0.33])
ma1 = np.array([1, 0.9])
ARMA_1_1 = ArmaProcess(ar1, ma1).generate_sample(nsample=1000)

In [ ]:
from itertools import product
ps = range(0, 8, 1)
qs = range(0, 8, 1)
order_list = list(product(ps, qs))

In [ ]:
from typing import Union
from tqdm.notebook import tqdm
from statsmodels.tsa.statespace.sarimax import SARIMAX
def optimize_ARMA(endog: Union[pd.Series, list], order_list: list):
    results = []
    for order in tqdm(order_list):
        try:
            model = SARIMAX(endog, order=(order[0], 0, order[1]),simple_differencing=False).fit(disp=False)
        except:
            continue
        aic = model.aic
        results.append([order, aic])
    result_df = pd.DataFrame(results)
    result_df.columns = ['(p,q)', 'AIC']
    #Sort in ascending order, lower AIC is better
    result_df = result_df.sort_values(by='AIC',
        ascending=True).reset_index(drop=True)
    return result_df

In [ ]:
result_df = optimize_ARMA(train_subset[column_y], order_list) #A
result_df

# Seasonal

With time series decomposition, we can clearly identify and visualize the seasonal component of a time series. Here, we decompose the dataset for air passengers using the STL function from the statsmodels library.

In [ ]:
train_set = train_df[(train_df.country == 'Norway') & (train_df.store == 'KaggleMart') & (train_df['product'] == 'Kaggle Hat')].copy()

In [ ]:
from statsmodels.tsa.seasonal import STL
decomposition = STL(train_set[column_y], period=12).fit()
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True,
figsize=(10,8))
ax1.plot(decomposition.observed)
ax1.set_ylabel('Observed')
ax2.plot(decomposition.trend)
ax2.set_ylabel('Trend')
ax3.plot(decomposition.seasonal)
ax3.set_ylabel('Seasonal')
ax4.plot(decomposition.resid)
ax4.set_ylabel('Residuals')
fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
ps = range(0, 13, 1)
qs = range(0, 13, 1)
Ps = [0]
Qs = [0]
d = 2
D = 0
s = 12
ARIMA_order_list = list(product(ps, qs, Ps, Qs))

In [ ]:
from typing import Union
from tqdm.notebook import tqdm

from statsmodels.tsa.statespace.sarimax import SARIMAX
def optimize_SARIMA(endog: Union[pd.Series, list], order_list: list, d: int, D: int, s: int) -> pd.DataFrame:
    results = []
    for order in tqdm(order_list):
        try:
            model = SARIMAX(
                endog,
                order=(order[0], d, order[1]),
                seasonal_order=(order[2], D, order[3], s),
                simple_differencing=False).fit(disp=False)
        except:
            continue
        aic = model.aic
        results.append([order, model.aic])
        
    result_df = pd.DataFrame(results)
    result_df.columns = ['(p,q,P,Q)', 'AIC']
    #Sort in ascending order, lower AIC is better
    result_df = result_df.sort_values(by='AIC',
        ascending=True).reset_index(drop=True)
    return result_df 

In [ ]:
train = train_subset[column_y]
ARIMA_result_df = optimize_SARIMA(train, ARIMA_order_list, d, D, s) #B
ARIMA_result_df

In [ ]:
ARIMA_model = SARIMAX(train, order=(9,11,0), simple_differencing=False)
ARIMA_model_fit = ARIMA_model.fit(disp=False)
ARIMA_model_fit.plot_diagnostics(figsize=(12,10))

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
residuals = ARIMA_model_fit.resid
lbvalue, pvalue = acorr_ljungbox(residuals, np.arange(1, 11, 1))
print(pvalue)